In [1]:
#This code was made by Hideo NAKANO to map epitope along with the protein sequence of target 
#based on enrithment ration of three amino acid combination calculated from NGS analysis
# with the help of ChatGPT 20230725
# modified to remove the surrounding amino acids
# changed to 2 aa combinations 20230725

import pandas as pd  # Data manipulation and analysis
import matplotlib.pyplot as plt  # Plotting library
import math  # Mathematical functions
from Bio import SeqIO  # For reading and writing sequence file formats
import csv  # For working with CSV files


# Function to calculate ratio
def calc_ratio(pep, ngs_sum_list, count_sum):
    # Initialize counters
    count_a = 0  
    count_p = 0  
    # Iterate over each item in list
    for item in ngs_sum_list:
        # If the peptide is found in the item's 2nd position
        if pep in item[1]:  
            count_p += item[2]  # Increment peptide count
            count_a += 1  # Increment count
    # Calculate ratio
    count_r = count_p/count_sum  
    # Return a list with peptide, ratio and count
    return [pep, count_r, count_a]  

# Function to enrich peptide
def enrich_peptide(pep_list_list, ngs_sum_list_ori, ngs_sum_list_r1, count_sum_ori, count_sum_r1):
    # Initialize output lists
    out_ori = [["peptide", "ratio_sum_ori", "count_ori"]]  
    # Extend the list with the calculated ratios
    out_ori.extend(list(map(lambda pep: calc_ratio(pep, ngs_sum_list_ori, count_sum_ori), pep_list_list)))  

    # Similar steps for another output list
    out_r1 = [["peptide", "ratio_sum_r1", "count_ori"]]  
    out_r1.extend(list(map(lambda pep: calc_ratio(pep, ngs_sum_list_r1, count_sum_r1), pep_list_list)))  

    # Final output list
    out=[["peptide", "EF", "ratio_ori", "ratio_r1"]]  
    for i in range(1,len(out_ori)):
        if out_ori[i][1] > 0:  # Check if the ratio is greater than 0
            EF = out_r1[i][1]/out_ori[i][1]  # Calculate enrichment factor
            out.append([out_ori[i][0],EF,out_ori[i][1],out_r1[i][1]])  # Append to the output list
        else:
            out.append([out_ori[i][0],'NaN',out_ori[i][1],out_r1[i][1]])  # Append 'NaN' if ratio is 0
    return out  # Return the output list

# Main function where the program starts
def main():
    # Specify names of the CSV files and plot
    csv_name = "epitope_out_400_R1.csv"  #output file of enrichment factor of  three amino acid epitope 
    pep_list_name = "ep_pep_list400.csv" #input file of amino acid combination list 
    ngs_sum_ori_name = "NostarX_Round0.csv" # Summary of the initial NGS data o
    ngs_sum_r1_name = "NostarX_Round3.csv"  #Summnary of the NGS data after selection
    fasta_target = list(SeqIO.read("HA-NEW.fa", "fasta"))  # Read target fasta sequence.
    csv_name_target= "antiHAtag_R1_2aa.csv"  #CSV output of the target seq
    plot_name_target = "antiHAtag_R1_2aa.jpeg" # Plot file of the target  

    # Load data from CSV files into pandas DataFrames
    ngs_sum_ori = pd.read_csv(ngs_sum_ori_name, keep_default_na=False, na_values=['_'])   
    ngs_sum_r1 = pd.read_csv(ngs_sum_r1_name, keep_default_na=False, na_values=['_'])
    pep_list = pd.read_csv(pep_list_name, keep_default_na=False, na_values=['_'])



    
    ngs_sum_ori['pep'] =  ngs_sum_ori.iloc[:,1]  
    ngs_sum_r1['pep'] =  ngs_sum_r1.iloc[:,1] 

    # Calculate the sum of column 2 in both DataFrames
    count_sum_ori = ngs_sum_ori.iloc[:,2].sum()  
    count_sum_r1 = ngs_sum_r1.iloc[:,2].sum()  

    # Convert the data in DataFrame to list format
    pep_list_list = pep_list.values.flatten().tolist()  
    ngs_sum_list_ori = ngs_sum_ori.values.tolist()  
    ngs_sum_list_r1 = ngs_sum_r1.values.tolist()  

    # Call the function enrich_peptide to calculate enrichment
    out = enrich_peptide(pep_list_list, ngs_sum_list_ori, ngs_sum_list_r1, count_sum_ori, count_sum_r1)  

    # Write the output to a CSV file
    with open(csv_name, "w", newline="") as f:  
        writer = csv.writer(f)  
        writer.writerows(out)  

    # Load the CSV file into a DataFrame
    pep_enrich = pd.read_csv(csv_name, keep_default_na=False, na_values=['_'])  

    # Read a fasta sequence
    
    # Break the sequence into sub-sequences of length 2
    tar_seq = [''.join(fasta_target[i:i+2]) for i in range(len(fasta_target)-1)]  

    # Create a new DataFrame
    df = pd.DataFrame({'Seq': fasta_target, 'EF': [0]*len(fasta_target)})  

    # Update the 'EF' column in DataFrame based on 'peptide' column in another DataFrame
    for i, item in enumerate(tar_seq):
        if i>=len(tar_seq):
            break
        df.loc[i:i+1, 'EF'] += pep_enrich.loc[pep_enrich['peptide'].str.contains(item), 'EF'].sum()  

    # Save the DataFrame to a CSV file
    df.to_csv(csv_name_target)  

    # Calculate length for the plot
    x_len = math.sqrt(len(fasta_target))*4+5  
    plt.rcParams["figure.figsize"] = [x_len,4.0]  
    # Plot the data
    df.plot(kind='bar',x='Seq', width=1)  
    plt.xticks(rotation=0)  
    plt.savefig(plot_name_target)  # Save the plot as a JPEG file
    plt.show()  # Display the plot

# Run the main function
if __name__ == "__main__":  
    main()  


<Figure size 7755.34x400 with 1 Axes>